* 函式名稱：image_crawler

* 函式參考來源：   https://medium.com/%E4%BC%81%E9%B5%9D%E4%B9%9F%E6%87%82%E7%A8%8B%E5%BC%8F%E8%A8%AD%E8%A8%88/python%E7%88%AC%E8%9F%B2-python-selenium-%E8%87%AA%E5%8B%95%E5%8C%96%E7%88%AC%E5%8F%96%E5%A4%A7%E9%87%8F%E5%9C%96%E7%89%87-a35d3c89c6d1

* 函式作者：陳宇鑫
* 注意事項：
    * 必須先下載ChromeDriver的檔案(執行檔)，並放在和該程式碼同一個目錄下
    * 程式碼在跑的過程中，不要動到新開啟的Chrome視窗(受到自動測試軟體的控制的那個)
* 函式參數：
    * target_name：目標照片主題的名字
    * url：爬取頁面網址
    * img_num = 100：要爬取的照片數量
    * base_path = 'imgs'：儲存圖片的基本路徑
    * chromeDriver = 'chromedriver'：ChromeDriver檔案存放的路徑(預設是和該程式同一個目錄下)

In [1]:
! pip install selenium

In [2]:
from selenium import webdriver
import time
import urllib
import os

def image_crawler(target_name, url, img_num = 100, base_path = 'imgs', chromeDriver = 'chromedriver'):
    print('----------- ChromeDriver準備中 -----------')
    
    # 目標元素的xpath
    xpath = '//div[@id="imgid"]/ul/li/a/img'
    
    # 啟動chrome瀏覽器
    driver = webdriver.Chrome(chromeDriver) 

    # 最大化窗口，因為每一次爬取只能看到視窗内的圖片  
    driver.maximize_window()  

    # 紀錄下載過的圖片網址，避免重複下載  
    img_url_dic = {}  

    # 瀏覽器打開爬取頁面
    driver.get(url)  

    # 如果沒有存圖的資料夾，則自動創建
    local_path = os.path.join(base_path, target_name)
    if not os.path.exists(local_path):
            os.makedirs(local_path)
            print('----------- 建立新資料夾：' + local_path + ' -----------')

    # 模擬滾動視窗瀏覽更多圖片
    pos = 0  
    m = 0 # 圖片編號 
    
    print('----------- 開始爬取圖片 -----------')
    for i in range(100):  
        pos += i*500 # 每次下滾500  
        js = "document.documentElement.scrollTop=%d" % pos  
        driver.execute_script(js)  
        time.sleep(2) # 設定休息時間，避免過度爬取被網頁擋掉(爬蟲必備的小技巧)

        for element in driver.find_elements_by_xpath(xpath):

            img_url = element.get_attribute('src')

            # 保存圖片到指定路徑
            if img_url != None and not img_url in img_url_dic:
                img_url_dic[img_url] = ''  
                m += 1

                # 如果已經儲存一定數量的照片，提前結束該函式
                if m > img_num:
                    driver.close()
                    print('---------- 程式處理完成，一共取得' + str(img_num) + '張' + target_name + '的圖片 ----------')
                    return

                # print(img_url)
                ext = img_url.split('/')[-1]
                # print(ext)
                filename = target_name + '_' + str(m).zfill(3) + '.jpg' # zfill可以字串未滿的位數補0，例如001、002...099、100

                # 每保存10張照片印出進度
                if m % 10 == 0:
                    print('目前進度：正在保存', filename)

                # 保存圖片
                urllib.request.urlretrieve(img_url, os.path.join(local_path, filename))

    # 結束該函式
    driver.close()
    print('---------- 程式處理完成，一共取得' + str(img_num) + '張' + target_name + '的圖片 ----------')
    return

In [7]:
### 讀取歌手名單 ###
with open('歌手名單_剩餘.txt', 'r', encoding="utf-8") as f:
    singer_list = f.read().splitlines()
print(singer_list) # 印出來看名單是否無誤

['阿信 五月天', '阿璞 八三夭', 'HipHopMan歐陽靖', 'PG One', '邱勝翊 王子', '陳奕迅', '小宇 宋念宇', '王力宏', '青峰 蘇打綠']


In [4]:
!pip install opencc-python-reimplemented

In [8]:
### 生成簡體版本的歌手名單，以便之後擷取藝人圖片搜尋結果的網址 ###
from opencc import OpenCC
cc = OpenCC('t2s')
singer_list_simplified = [cc.convert(singer) for singer in singer_list]
print(singer_list_simplified) # 印出來看名單是否無誤

['阿信 五月天', '阿璞 八三夭', 'HipHopMan欧阳靖', 'PG One', '邱胜翊 王子', '陈奕迅', '小宇 宋念宇', '王力宏', '青峰 苏打绿']


In [9]:
### 主函式 ###
for idx, singer_name in enumerate(singer_list):
    # 目標照片主題的名字
    print('---------------------- 目前進度：' + singer_name + ' ----------------------')
    target_name = singer_name

    # 爬取頁面網址 
    url = 'https://pic.sogou.com/pics?query=' + singer_list_simplified[idx]
    
    # 要爬取照片的數量
    # (之後將人工篩掉50張比較不良的照片，例如重複的照片、非正臉的照片、根本不是本人的照片、太多人同框的照片、背景佔太多的照片...)
    img_num = 200

    # 儲存圖片的基本路徑
    base_path = 'imgs'

    # ChromeDriver檔案放的位置
    chromeDriver = 'chromedriver' # 預設是和該程式同一個目錄下
    
    image_crawler(target_name, url, img_num, base_path, chromeDriver)

---------------------- 目前進度：阿信 五月天 ----------------------
----------- ChromeDriver準備中 -----------
----------- 建立新資料夾：imgs\阿信 五月天 -----------
----------- 開始爬取圖片 -----------
目前進度：正在保存 阿信 五月天_010.jpg
目前進度：正在保存 阿信 五月天_020.jpg
目前進度：正在保存 阿信 五月天_030.jpg
目前進度：正在保存 阿信 五月天_040.jpg
目前進度：正在保存 阿信 五月天_050.jpg
目前進度：正在保存 阿信 五月天_060.jpg
目前進度：正在保存 阿信 五月天_070.jpg
目前進度：正在保存 阿信 五月天_080.jpg
目前進度：正在保存 阿信 五月天_090.jpg
目前進度：正在保存 阿信 五月天_100.jpg
目前進度：正在保存 阿信 五月天_110.jpg
目前進度：正在保存 阿信 五月天_120.jpg
目前進度：正在保存 阿信 五月天_130.jpg
目前進度：正在保存 阿信 五月天_140.jpg
目前進度：正在保存 阿信 五月天_150.jpg
目前進度：正在保存 阿信 五月天_160.jpg
目前進度：正在保存 阿信 五月天_170.jpg
目前進度：正在保存 阿信 五月天_180.jpg
目前進度：正在保存 阿信 五月天_190.jpg
目前進度：正在保存 阿信 五月天_200.jpg
---------- 程式處理完成，一共取得200張阿信 五月天的圖片 ----------
---------------------- 目前進度：阿璞 八三夭 ----------------------
----------- ChromeDriver準備中 -----------
----------- 建立新資料夾：imgs\阿璞 八三夭 -----------
----------- 開始爬取圖片 -----------
目前進度：正在保存 阿璞 八三夭_010.jpg
目前進度：正在保存 阿璞 八三夭_020.jpg
目前進度：正在保存 阿璞 八三夭_030.jpg
目前進度：正在保存 阿璞 八三夭_040.jpg
目前進度：正在保存

失敗案例：google_images_download (以下程式碼為我之前實驗失敗的案例，不用跑以下的程式碼)

In [ ]:
# ! pip install google_images_download

In [ ]:
# ! googleimagesdownload -k '羅志祥' -l 100

In [ ]:
# from google_images_download import google_images_download
# #instantiate the class
# response = google_images_download.googleimagesdownload()
# arguments = {"keywords":"lilly,hills","limit":5,"print_urls":True}
# paths = response.download(arguments)
# #print complete paths to the downloaded images
# print(paths)